# Db2 11.5.4 RESTful Programming
The following notebook is a brief example of how to use the Db2 11.5.4 RESTful Endpoint service to extend the capabilies of Db2.

Programmers can create Representational State Transfer (REST) endpoints that can be used to interact with Db2.

Each endpoint is associated with a single SQL statement. Authenticated users of web, mobile, or cloud applications can use these REST endpoints from any REST HTTP client without having to install any Db2 drivers.

The Db2 REST server accepts an HTTP request, processes the request body, and returns results in JavaScript Object Notation (JSON).

The Db2 REST server is pre-installed and running on Docker on host3 (10.0.0.4) in the Demonstration cluster. As a programmer you can communicate with the service on port 50050. Your welcome note includes the external port you can use to interact with the Db2 RESTful Endpoint service directly.

You can find more information about this service at: https://www.ibm.com/support/producthub/db2/docs/content/SSEPGG_11.5.0/com.ibm.db2.luw.admin.rest.doc/doc/c_rest.html.

### Finding the Db2 RESTful Endpoint Service API Documentation
If you are running this notebook from a browser running inside the Cloud Pak for Data cluster, click: http://10.0.0.4:50050/docs If you are running this from a browser from your own desktop, check your welcome note for the address of the Db2 RESTful Service at port 50050.

## Getting Started
Before you can start submitting SQL or creating your own services you need to complete a few setup steps. 

### Import the required programming libraries
The requests library is the minimum required by Python to construct RESTful service calls. The Pandas library is used to format and manipulate JSON result sets as tables.

In [ ]:
import requests
import pandas as pd

### Create the Header File required for getting an authetication token
We have to provide the location of the RESTful service for our calls.
The RESTful call to the Db2 RESTful Endpoint service is contructed and transmitted as JSON. The first part of the JSON structure is the headers that define the content tyoe of the request.

In [ ]:
headers = {
  "content-type": "application/json"
}

### Define the RESTful Host
The next part defines where the request is sent to. It provides the location of the RESTful service for our calls.

In [ ]:
Db2RESTful = "http://localhost:50050"

### API Authentication Service
Each service has its own path in the RESTful call. For authentication we need to point to the `v1/auth` service.

In [ ]:
API_Auth = "/v1/auth"

### Database Connection Information
To authenticate to the RESTful service you must provide the connection information for the database along with the userid and password that you are using to authenticate with. You can also provide an expiry time so that the access token that gets returned will be invalidated after that time period.

In [ ]:
body = {
  "dbParms": {
    "dbHost": "10.0.0.1",
    "dbName": "ONTIME",
    "dbPort": 50001,
    "isSSLConnection": False,
    "username": "db2inst1",
    "password": "db2inst1"
  },
  "expiryTime": "8760h"
}

### Retrieving an Access Token
When communicating with the RESTful service, you must provide the name of the service that you want to interact with. In this case the authentication service is */v1/auth*. 

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_Auth), headers=headers, json=body)
    print (response)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

A response code of 200 means that the authentication worked properly, otherwise the error that was generated is printed. The response includes a connection token that is reused throughout the rest of this lab. It ensures secure a connection without requiring that you reenter a userid and password with each request.

In [ ]:
if (response.status_code == 200):
  token = response.json()["token"]
  print("Token: {}".format(token))
else:
  print(response.json()["errors"])

### Creating a standard reusable JSON header
The standard header for all subsequent calls will use this format. It includes the access token.

In [ ]:
headers = {
  "authorization": f"{token}",
  "content-type": "application/json"
}

## Executing an SQL Statement
Before you try creating your own customer service endpoint, you can try using some of the built in services. These let you submit SQL statements in a variety of ways. 

Executing SQL requires a different service endpoint. In this case we will use "/services/execsql"

In [ ]:
API_execsql = "/v1/services/execsql"

In this example the code requests that the RESTful function waits until the command is complete.

In [ ]:
sql = \
"""
SELECT AC."TAIL_NUMBER", AC."MANUFACTURER", AC."MODEL", OT."FLIGHTDATE", OT."UNIQUECARRIER", OT."AIRLINEID", OT."CARRIER", OT."TAILNUM", OT."FLIGHTNUM", OT."ORIGINAIRPORTID", OT."ORIGINAIRPORTSEQID", OT."ORIGINCITYNAME", OT."ORIGINSTATE", OT."DESTAIRPORTID", OT."DESTCITYNAME", OT."DESTSTATE", OT."DEPTIME", OT."DEPDELAY", OT."TAXIOUT", OT."WHEELSOFF", OT."WHEELSON", OT."TAXIIN", OT."ARRTIME", OT."ARRDELAY", OT."ARRDELAYMINUTES", OT."CANCELLED", OT."AIRTIME", OT."DISTANCE"
  FROM "ONTIME"."ONTIME" OT, "ONTIME"."AIRCRAFT" AC 
  WHERE AC."TAIL_NUMBER" = OT.TAILNUM
  AND ORIGINSTATE = 'NJ'
  AND DESTSTATE = 'CA'
  AND AC.MANUFACTURER = 'Boeing' 
  AND AC.MODEL LIKE 'B737%'
  AND OT.TAXIOUT > 30
  AND OT.DISTANCE > 2000
  AND OT.DEPDELAY > 300
  ORDER BY OT.ARRDELAY;
"""

In [ ]:
body = {
  "isQuery": True,
  "sqlStatement": sql,
  "sync": True
}
print(body)

In [ ]:
def runStatement(sql, isQuery) :
    body = {
      "isQuery": isQuery,
      "sqlStatement": sql,
      "sync": True
    }
    try:
        response = requests.post("{}{}".format(Db2RESTful,API_execsql), headers=headers, json=body)
        return response
    except Exception as e:
        print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
response = runStatement(sql, True)

If the successful call returns a **200** response code.

In [ ]:
print(response)

Now that you know the call is a success, you can retrieve the json in the result set.

In [ ]:
print(response.json()["resultSet"])

To format the results, use a Pandas Dataframe class to convert the json result set into a table. Dataframes can be used to further manipulate results in Python.

In [ ]:
display(pd.DataFrame(response.json()['resultSet']))

## Use Parameters in a SQL Statement
Simple parameter passing is also available through the execsql service. In this case we are passing the employee number into the query to retrieve the full employee record. Try substituting different employee numbers and run the REST call again. For example, you can change "000010" to "000020", or "000030".

In [ ]:
sqlparm = \
"""
SELECT AC."TAIL_NUMBER", AC."MANUFACTURER", AC."MODEL", OT."FLIGHTDATE", OT."UNIQUECARRIER", OT."AIRLINEID", OT."CARRIER", OT."TAILNUM", OT."FLIGHTNUM", OT."ORIGINAIRPORTID", OT."ORIGINAIRPORTSEQID", OT."ORIGINCITYNAME", OT."ORIGINSTATE", OT."DESTAIRPORTID", OT."DESTCITYNAME", OT."DESTSTATE", OT."DEPTIME", OT."DEPDELAY", OT."TAXIOUT", OT."WHEELSOFF", OT."WHEELSON", OT."TAXIIN", OT."ARRTIME", OT."ARRDELAY", OT."ARRDELAYMINUTES", OT."CANCELLED", OT."AIRTIME", OT."DISTANCE"
  FROM "ONTIME"."ONTIME" OT, "ONTIME"."AIRCRAFT" AC 
  WHERE AC."TAIL_NUMBER" = OT.TAILNUM
  AND ORIGINSTATE = 'NJ'
  AND DESTSTATE = 'CA'
  AND AC.MANUFACTURER = 'Boeing' 
  AND AC.MODEL LIKE 'B737%'
  AND OT.TAXIOUT > 30
  AND OT.DISTANCE > 2000
  AND OT.DEPDELAY > ?
  ORDER BY OT.ARRDELAY;
"""

body = {
  "isQuery": True,
  "parameters" : {
      "1" : 300
  },
  "sqlStatement": sqlparm,
  "sync": True
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_execsql), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print(response)

In [ ]:
response.json()["resultSet"]

In [ ]:
display(pd.DataFrame(response.json()['resultSet']))

## Generate a Call and don't wait for the results
If you know that your statement will take a long time to return a result, you can check back later. Turn **sync** off to avoid waiting.

In [ ]:
sql = \
"""
SELECT AC."TAIL_NUMBER", AC."MANUFACTURER", AC."MODEL", OT."FLIGHTDATE", OT."UNIQUECARRIER", OT."AIRLINEID", OT."CARRIER", OT."TAILNUM", OT."FLIGHTNUM", OT."ORIGINAIRPORTID", OT."ORIGINAIRPORTSEQID", OT."ORIGINCITYNAME", OT."ORIGINSTATE", OT."DESTAIRPORTID", OT."DESTCITYNAME", OT."DESTSTATE", OT."DEPTIME", OT."DEPDELAY", OT."TAXIOUT", OT."WHEELSOFF", OT."WHEELSON", OT."TAXIIN", OT."ARRTIME", OT."ARRDELAY", OT."ARRDELAYMINUTES", OT."CANCELLED", OT."AIRTIME", OT."DISTANCE"
  FROM "ONTIME"."ONTIME" OT, "ONTIME"."AIRCRAFT" AC 
  WHERE AC."TAIL_NUMBER" = OT.TAILNUM
  AND ORIGINSTATE = 'NJ'
  AND DESTSTATE = 'CA'
  AND AC.MANUFACTURER = 'Boeing' 
  AND AC.MODEL LIKE 'B737%'
  AND OT.TAXIOUT > 30
  AND OT.DISTANCE > 2000
  AND OT.DEPDELAY > 300
  ORDER BY OT.ARRDELAY;
"""

body = {
  "isQuery": True,
  "sqlStatement": sql,
  "sync": False
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_execsql), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print(response)

Retrieve the job id, so that you can retrieve the results later.

In [ ]:
job_id = response.json()["id"]

In [ ]:
print(job_id)

## Retrieve Result set using Job ID
The service API needs to be appended with the Job ID.

In [ ]:
API_get = "/v1/services/"

We can limit the number of rows that we return at a time. Setting the limit to zero means all of the rows are to be returned.

In [ ]:
body = {
  "limit": 0
}

Get the results.

In [ ]:
try:
    response = requests.get("{}{}{}".format(Db2RESTful,API_get,job_id), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print(response)

Retrieve the results.

In [ ]:
display(pd.DataFrame(response.json()['resultSet']))

Now that you have some experience with the built in SQL service, you can try creating your own endpoint service. 

## Using RESTful Endpoint Services
The most common way of interacting with the service is to fully encapsulate an SQL statement, including any parameters, in a unique RESTful service. This creates a secure separation between the database service and the RESTful programming service. It also allows you to create versions of the same service to make maintenance and evolution of programming models simple and predictable.

### Setup the Meta Data Tables and Stored Procedures to manage Endpoint Services
Before you can start defining and running your own RESTful Endpoint services you need call the service to create the table and stored procedures in the database you are using. 

In [ ]:
API_makerest = "/v1/metadata/setup"

You can specify the schema that the new table and stored procedures will be created in. In this example we will use **DB2REST**

In [ ]:
body = {
  "schema": "DB2REST"
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_makerest), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

If the process is successful the service returns a 201 status code.

In [ ]:
if (response.status_code == 201):
  print(response.reason)
else:
  print(response.json())

### Create a RESTful Service
Now that the RESTful Service metadata is created in your database, you can create your first service. In this example you will pass an employee numb er, a 6 character string, to the service. It will return the department number of the employee.

In [ ]:
API_makerest = "/v1/services"

The first step is to define the SQL that we want in the RESTful call. Parameters are identified using an ampersand "@". Notice that our SQL is nicely formatted to make this notebook easier to ready. However when creating a service it is good practice to remove the line break characters from your SQL statement. 

In [ ]:
sql = \
"""
SELECT COUNT(AC."TAIL_NUMBER") FROM "ONTIME"."ONTIME" OT, "ONTIME"."AIRCRAFT" AC 
  WHERE AC."TAIL_NUMBER" = OT.TAILNUM
  AND ORIGINSTATE = @STATE
  AND DESTSTATE = 'CA'
  AND AC.MANUFACTURER = 'Boeing' 
  AND AC.MODEL LIKE 'B737%'
  AND OT.TAXIOUT > 30
  AND OT.DISTANCE > 2000
  AND OT.DEPDELAY > @DELAY
  FETCH FIRST 5 ROWS ONLY
"""
sql = sql.replace("\n","")

The next step is defining the jason body to send along with the REST call.

In [ ]:
body = {"isQuery": True,
       "parameters": [
         {
         "datatype": "CHAR(2)",
         "name": "@STATE"
         },
         {
         "datatype": "INT",
         "name": "@DELAY"    
         }
       ],
       "schema": "DEMO",
       "serviceDescription": "Delay",
       "serviceName": "delay",
       "sqlStatement": sql,
       "version": "1.0"
}

Now submit the full RESTful call to create the new service. 

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_makerest), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print(response)

### Call the new RESTful Service
Now you can call the RESTful service. In this case we will pass the stock symbol CAT. But like in the previous example you can try rerunning the service call with different stock symbols.

In [ ]:
API_runrest = "/v1/services/delay/1.0"

In [ ]:
body = {
  "parameters": {
    "@STATE": "NY","@DELAY":"300"
  },
  "sync": True
}

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_runrest), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print("{}{}".format(Db2RESTful,API_runrest))

In [ ]:
print(response)

In [ ]:
print(response.json())

You can retrieve the result set, convert it into a Dataframe and display the table.

In [ ]:
display(pd.DataFrame(response.json()['resultSet']))

## Loop through the new call
Now you can call the RESTful service with different values.

In [ ]:
API_runrest = "/v1/services/delay/1.0"

In [ ]:
repeat = 2
for x in range(0, repeat):
    for state in ("OH", "NJ", "NY", "FL", "MI"):

        body = {
          "parameters": {
            "@STATE": state,"@DELAY": "240"
          },
          "sync": True
        }
        try:
            response = requests.post("{}{}".format(Db2RESTful,API_runrest), headers=headers, json=body)
            print(state + ": " + str(response.json()['resultSet']))
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))

## Managing Your Services 
There are several service calls you can use to help manage the Db2 RESTful Endpoint service. 

## List Available Services
You can also list all the user defined services you have access to

In [ ]:
API_listrest = "/v1/services"

In [ ]:
try:
    response = requests.get("{}{}".format(Db2RESTful,API_listrest), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print(response.json())

In [ ]:
display(pd.DataFrame(response.json()['Db2Services']))

## Get Service Details
You can also get the details of a service

In [ ]:
API_getDetails = "/v1/services/delay/3.0"

In [ ]:
try:
    response = requests.get("{}{}".format(Db2RESTful,API_getDetails), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
json = response.json()
print(json)

You can format the result to make it easier to ready. For example, here are the input and outputs.

In [ ]:
display(pd.DataFrame(json['inputParameters']))
display(pd.DataFrame(json['resultSetFields']))

## Delete a Service
A single call is also available to delete a service

In [ ]:
API_deleteService = "/v1/services"
Service = "/delay"
Version = "/1.0"

In [ ]:
try:
    response = requests.delete("{}{}{}{}".format(Db2RESTful,API_deleteService,Service,Version), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
print (response)

## Get Service Logs
You can also easily download the Db2 RESTful Endpoint service logs. 

In [ ]:
API_listrest = "/v1/logs"

In [ ]:
try:
    response = requests.get("{}{}".format(Db2RESTful,API_listrest), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

In [ ]:
if (response.status_code == 200):
  myFile = response.content
  open('/tmp/logs.zip', 'wb').write(myFile)
  print("Downloaded",len(myFile),"bytes.")
else:
  print(response.json())

To see the content of the logs, open the Files browser on machine host3 (10.0.0.4). Navigate to the **/tmp** directory and unzip the logs file. 

## Using the Db2 REST Class

In [ ]:
# Run the Db2REST Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
import json
import requests
import pandas as pd

class Db2REST():
    
    def __init__(self, RESTServiceURL):
        self.headers = {"content-type": "application/json"}
        self.RESTServiceURL = RESTServiceURL
        self.version = "/v1"
        self.API_auth = self.version + "/auth"
        self.API_makerest = self.version + "/metadata/setup"
        self.API_services = self.version + "/services/"       
        self.API_version = self.version + "/version/"      
        self.API_execsql = self.API_services + "execsql"
        self.API_monitor = self.API_services + "monitor" 
        
    def connectDatabase(self, dbHost, dbName, dbPort, isSSLConnection, dbUsername, dbPassword, expiryTime="300m"):
        self.dbHost = dbHost
        self.dbName = dbName
        self.dbPort = dbPort
        self.isSSLConnection = isSSLConnection
        self.dbusername = dbUsername
        self.dbpassword = dbPassword      
        self.connectionBody = {
            "dbParms": {
            "dbHost": dbHost,
            "dbName": dbName,
            "dbPort": dbPort,
            "isSSLConnection": isSSLConnection,
            "username": dbUsername,
            "password": dbPassword
            },
            "expiryTime": expiryTime
        }
        try:
            response = requests.post("{}{}".format(self.RESTServiceURL,self.API_auth), headers=self.headers, json=self.connectionBody)
            print (response)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            
        if (response.status_code == 200):
            self.token = response.json()["token"]
            print("Successfully connected and retrieved access token")
        else:
            print(response.json()["errors"])
    
        self.headers = {
            "authorization": f"{self.token}",
            "content-type": "application/json"
        }
        
    def getConnection(self):
        return self.connectionBody
    
    def getService(self):
        return self.RESTServiceURL
    
    def getToken(self):
        return("Token: {}".format(self.token))
    
    def getVersion(self):
        try:
            response = requests.get("{}{}".format(self.RESTServiceURL,self.API_version), headers=self.headers)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
         
        if (response.status_code == 200):
            return response.json()['version']
        else:
            print(response.json()['errors'][0]['more_info'])        
        
    def runStatement(self, sql, isQuery=True, sync=True, parameters={}):
        body = {
            "isQuery": isQuery,
            "sqlStatement": sql,
            "sync": sync,
            "parameters": parameters
        }
        
        try:
            response = requests.post("{}{}".format(self.RESTServiceURL,self.API_execsql), headers=self.headers, json=body)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
         
        if (response.status_code == 200):
            return pd.DataFrame(response.json()['resultSet'])
        elif (response.status_code == 202):
            return response.json()["id"]
        else:
            print(response.json()['errors'][0]['more_info'])
            
    def getResult(self, job_id, limit=0):
        body = {"limit": limit}
        
        try:
            response = requests.get("{}{}{}".format(self.RESTServiceURL,self.API_services,job_id), headers=self.headers, json=body)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
  
        if (response.status_code == 200):
            json = response.json()   
            if (json['jobStatus'] == 2):
                return json['jobStatusDescription']
            elif (json['jobStatus'] == 3):
                return pd.DataFrame(json['resultSet'])               
            elif (json['jobStatus'] == 4):
                return pd.DataFrame(json['resultSet'])  
            else: 
                return json
        elif (response.status_code == 404):
            print(response.json()['errors'])  
        elif  (response.status_code == 500):
            print(response.json()['errors'][0]['more_info'])            
        else:
            print(response.json())
            
    def createServiceMetadata(self, serviceSchema="Db2REST"):
        self.serviceSchema = serviceSchema
        body = {"schema": self.serviceSchema}
        try:
            response = requests.post("{}{}".format(self.RESTServiceURL,self.API_makerest), headers=self.headers, json=body)
            if (response.status_code == 201):
                print(response.reason)
            else:
                print(response.json())
            
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            

    def listServices(self):
        try:
            response = requests.get("{}{}".format(self.RESTServiceURL,self.API_services), headers=self.headers)
            return pd.DataFrame(response.json()['Db2Services'])
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            
    def getServiceDetails(self, serviceName, version):
        try:
            response = requests.get("{}{}{}{}".format(self.RESTServiceURL,self.API_services,"/" + serviceName,"/" + version), headers=self.headers)
            print(response.status_code)
            if (response.status_code == 200):
                description = response.json()
                print("Input parameters:")
                print(description["inputParameters"])
                print("Result format:")
                print(description["resultSetFields"])
            else:
                print(response.json())        
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))            
            
    def createService(self, schema, serviceDescription, serviceName, sql, version, parameters=False, isQuery=True):
        if (parameters==False):
            body = {"isQuery": isQuery,
                "schema": schema,
                "serviceDescription": serviceDescription,
                "serviceName": serviceName,
                "sqlStatement": sql.replace("\n",""),
                "version": version
            } 
        else: 
            body = {"isQuery": isQuery,
                "schema": schema,
                "serviceDescription": serviceDescription,
                "serviceName": serviceName,
                "sqlStatement": sql.replace("\n",""),
                "version": version,
                "parameters": parameters
            } 
        
        try:
            response = requests.post("{}{}".format(self.RESTServiceURL,self.API_services), headers=self.headers, json=body)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            
        if (response.status_code == 201):
             print("Service: " + serviceName + " Version: " + version + " created")
        else:
            print(response.json())  
            
    def deleteService(self, serviceName, version):
        try:
            response = requests.delete("{}{}{}{}".format(self.RESTServiceURL,self.API_services,"/" + serviceName,"/" + version), headers=self.headers)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            
        if (response.status_code == 204):
            print("Service: " + serviceName + " Version: " + version + " deleted")
        else:
            print(response.json())   
            
    def callService(self, serviceName, version, parameters, sync=True):
        body = {
            "parameters": parameters,
            "sync": sync
        }
        try:
            response = requests.post("{}{}{}{}".format(self.RESTServiceURL,self.API_services,"/" + serviceName,"/" + version), headers=self.headers, json=body)
            if (response.status_code == 200):
                return pd.DataFrame(response.json()['resultSet'])
            elif (response.status_code == 202):
                return response.json()["id"]
            else:
                print(response.json()['errors'][0]['more_info'])
                
        except Exception as e:
            if (repr(e) == "KeyError('more_info',)"): 
                print("Service not found")
            else: 
                print("Unable to call RESTful service. Error={}".format(repr(e)))
                
    def monitorJobs(self):
        try:
            response = requests.get("{}{}".format(self.RESTServiceURL,self.API_monitor), headers=self.headers)
            if (response.status_code == 200):
                return pd.DataFrame(response.json()['MonitorServices'])
            else:
                print(response.json())    
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))                       

### Setting up a Db2 RESTful Endpoint Service Class instance

In [ ]:
Db2RESTService = Db2REST("http://localhost:50050")
print("Db2 RESTful Endpoint Service Version: " + Db2RESTService.getVersion())

#### Connecting to the service to the database

In [ ]:
Db2RESTService.connectDatabase("10.0.0.1", "ONTIME", 50001, False, "db2inst1", "db2inst1")

#### Confirming the service settings

In [ ]:
print(Db2RESTService.getService())

#### Retrieve the database connection details

In [ ]:
print(Db2RESTService.getConnection())

#### Retrieve the access Token

In [ ]:
print(Db2RESTService.getToken())

### Running SQL Through the Service
You can run an SQL Statement through the RESTful service as a simple text string.

Let's start by defining the SQL to run:

In [ ]:
sql = \
"""
SELECT AC."TAIL_NUMBER", AC."MANUFACTURER", AC."MODEL", OT."FLIGHTDATE", OT."UNIQUECARRIER", OT."AIRLINEID", OT."CARRIER", OT."TAILNUM", OT."FLIGHTNUM", OT."ORIGINAIRPORTID", OT."ORIGINAIRPORTSEQID", OT."ORIGINCITYNAME", OT."ORIGINSTATE", OT."DESTAIRPORTID", OT."DESTCITYNAME", OT."DESTSTATE", OT."DEPTIME", OT."DEPDELAY", OT."TAXIOUT", OT."WHEELSOFF", OT."WHEELSON", OT."TAXIIN", OT."ARRTIME", OT."ARRDELAY", OT."ARRDELAYMINUTES", OT."CANCELLED", OT."AIRTIME", OT."DISTANCE"
  FROM "ONTIME"."ONTIME" OT, "ONTIME"."AIRCRAFT" AC 
  WHERE AC."TAIL_NUMBER" = OT.TAILNUM
  AND ORIGINSTATE = 'NJ'
  AND DESTSTATE = 'CA'
  AND AC.MANUFACTURER = 'Boeing' 
  AND AC.MODEL LIKE 'B737%'
  AND OT.TAXIOUT > 30
  AND OT.DISTANCE > 2000
  AND OT.DEPDELAY > 300
  ORDER BY OT.DEPDELAY DESC
  FETCH FIRST 5 ROWS ONLY;
"""

Now a single call to the **runStatement** routine runs the SQL synchronously and returns the result as a DataFrame

In [ ]:
result = (Db2RESTService.runStatement(sql))
display(result)

You can also run the statement asynchronously so you don't have to wait for the result. In this case the result is the statement identifier that you can use to check the statement status.

In [ ]:
statementID = (Db2RESTService.runStatement(sql, sync=False))
display(statementID)

If you have several statements running at the same time you can check to see their status with the **monitorStatus** routine and see where they are in the service queue. 

In [ ]:
services = Db2RESTService.monitorJobs()
display(services)

You can try to get the results of the statment by passing the statement identifier into the getResults routine. If the statement has finished running it will return a result set as a DataFrame. It is still running, a message is returned.

In [ ]:
result = (Db2RESTService.getResult(statementID))
display(result)

#### Passing Parameters when running SQL Statements

In [ ]:
sqlparm = \
"""
SELECT AC."TAIL_NUMBER", AC."MANUFACTURER", AC."MODEL", OT."FLIGHTDATE", OT."UNIQUECARRIER", OT."AIRLINEID", OT."CARRIER", OT."TAILNUM", OT."FLIGHTNUM", OT."ORIGINAIRPORTID", OT."ORIGINAIRPORTSEQID", OT."ORIGINCITYNAME", OT."ORIGINSTATE", OT."DESTAIRPORTID", OT."DESTCITYNAME", OT."DESTSTATE", OT."DEPTIME", OT."DEPDELAY", OT."TAXIOUT", OT."WHEELSOFF", OT."WHEELSON", OT."TAXIIN", OT."ARRTIME", OT."ARRDELAY", OT."ARRDELAYMINUTES", OT."CANCELLED", OT."AIRTIME", OT."DISTANCE"
  FROM "ONTIME"."ONTIME" OT, "ONTIME"."AIRCRAFT" AC 
  WHERE AC."TAIL_NUMBER" = OT.TAILNUM
  AND ORIGINSTATE = ?
  AND DESTSTATE = ?
  AND AC.MANUFACTURER = 'Boeing' 
  AND AC.MODEL LIKE 'B737%'
  AND OT.TAXIOUT > 30
  AND OT.DISTANCE > 2000
  AND OT.DEPDELAY > ?
  ORDER BY OT.DEPDELAY DESC
  FETCH FIRST 10 ROWS ONLY;
"""

In [ ]:
result = Db2RESTService.runStatement(sqlparm,parameters={"1": 'NY', "2": 'CA', "3" : 300})
display(result)

In [ ]:
result = Db2RESTService.runStatement(sqlparm,parameters={"1": 'NJ', "2": 'CA', "3" : 200})
display(result)

#### Limiting Results

In [ ]:
statementID = Db2RESTService.runStatement(sqlparm, sync=False, parameters={"1": 'NJ', "2": 'CA', "3" : 200})
display(statementID)
result = (Db2RESTService.getResult(statementID))
display(result)

This time the **getResult** routine include a parameter to limit the result set to 5 rows. 

In [ ]:
result = (Db2RESTService.getResult(statementID, limit=5))
display(result)

The next cell retrieves the remaining rows.

In [ ]:
result = (Db2RESTService.getResult(statementID))
display(result)

After all the rows have been returned the job history is removed. If you try to retrieve the results for this statement now the service won't find it.

In [ ]:
result = (Db2RESTService.getResult(statementID))
display(result)

### Creating and Running Endpoint Services

In [ ]:
Db2RESTService.createServiceMetadata("DB2REST")

Let's start by defining the SQL statement. It can include parameters that have to be idenfied with an amersand "@".

In [ ]:
sql = \
"""
SELECT COUNT(AC."TAIL_NUMBER") FROM "ONTIME"."ONTIME" OT, "ONTIME"."AIRCRAFT" AC 
  WHERE AC."TAIL_NUMBER" = OT.TAILNUM
  AND ORIGINSTATE = @STATE
  AND DESTSTATE = 'CA'
  AND AC.MANUFACTURER = 'Boeing' 
  AND AC.MODEL LIKE 'B737%'
  AND OT.TAXIOUT > 30
  AND OT.DISTANCE > 2000
  AND OT.DEPDELAY > @DELAY
  FETCH FIRST 5 ROWS ONLY
"""

Now we can create the service, including the two parameters, using the **createService** routine. 

In [ ]:
parameters = [{"datatype": "CHAR(2)","name": "@STATE"},{"datatype": "INT","name": "@DELAY"}]
schema = 'DEMO'
serviceDescription = 'Delay'
serviceName = 'delay'
version = '2.0'

Db2RESTService.createService(schema, serviceDescription, serviceName, sql, version, parameters)

A call to the **listServices** routine confirms that you have created the new service. 

In [ ]:
services = Db2RESTService.listServices()
display(services)

You can also see the details for any service using the **getServiceDetails** routine.

In [ ]:
details = Db2RESTService.getServiceDetails("delay","2.0")
display(details)

You can all the new service using the **callService** routine. The parameters are passed into call using an array of values. By default the call is synchronous so you have to wait for the results. 

In [ ]:
serviceName = 'delay'
version = '2.0'
parameters = {"@STATE": "NJ","@DELAY":"200"}
result = Db2RESTService.callService(serviceName, version, parameters)
display(result)

You can also call the service asychronously, just like we did with SQL statements earlier. Notice the additional parameter **sync=False**. Since the cell below immediately checks the status of the job you can see it has been queued. 

In [ ]:
serviceName = 'delay'
version = '2.0'
parameters = {"@STATE": "NJ","@DELAY":"200"}
statementID = Db2RESTService.callService(serviceName, version, parameters, sync=False)
display(statementID)  
display(Db2RESTService.monitorJobs())

Run **monitorJobs** again to confirm that the endpoint service has completed the request.

In [ ]:
services = Db2RESTService.monitorJobs()
display(services)

And retrieve the result set.

In [ ]:
result = (Db2RESTService.getResult(statementID))
display(result)

You can also delete an existing endpoint service with a call to the **deleteService** routine.

In [ ]:
serviceName = 'delay'
version = '2.0'
Db2RESTService.deleteService(serviceName, version)

#### Using a service to query the Catalog
You can also think about creating services to explore the database catalog. For example, here is a service that accepts a schema as an input parameter and returns a list of tables in the schema. 

In [ ]:
sql = \
"""
SELECT TABSCHEMA, TABNAME, ALTER_TIME FROM SYSCAT.TABLES WHERE TABSCHEMA = @SCHEMA
"""

parameters = [{"datatype": "VARCHAR(64)","name": "@SCHEMA"}]
schema = 'DEMO'
serviceDescription = 'Tables'
serviceName = 'tables'
version = '1.0'

Db2RESTService.createService(schema, serviceDescription, serviceName, sql, version, parameters)

In [ ]:
serviceName = 'tables'
version = '1.0'
result = Db2RESTService.callService(serviceName, version, parameters = {"@SCHEMA": "SYSCAT"}, sync=True)
display(result)

## What's Next
Try experimenting. Create your own services. You can find out more at: https://www.ibm.com/support/producthub/db2/docs/content/SSEPGG_11.5.0/com.ibm.db2.luw.admin.rest.doc/doc/c_rest.html.

Also check out the OpenAPI specification for the service. It includes coding examples in Python, CURL and JavaScript. 

If you are running this notebook from a browser running inside the Cloud Pak for Data cluster, click: http://10.0.0.4:50050/docs If you are running this from a browser from your own desktop, check your welcome note for the address of the Db2 RESTful Service at port 50050 and add **docs** to the end of the URL.